# Resources:

[models](https://ai.google.dev/gemini-api/docs/models)


[console](https://console.cloud.google.com/apis/credentials)

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
from google import genai

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch your key.
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
# prompt: fetch a codebase from github, i want to iterate overall the files and organise it in a strutural way as it is present in the github and then pass it as a context to the model

import os
import subprocess

def clone_github_repo(repo_url, branch='main', local_path='repo'):
    """Clones a GitHub repository locally."""
    if os.path.exists(local_path):
        print(f"Directory '{local_path}' already exists. Skipping clone.")
        return local_path
    try:
        subprocess.run(['git', 'clone', '--depth', '1', '--branch', branch, repo_url, local_path], check=True)
        print(f"Repository cloned to '{local_path}'")
        return local_path
    except subprocess.CalledProcessError as e:
        print(f"Error cloning repository: {e}")
        return None

def organize_repo_structure_local(local_path):
    """Organizes local repository content into a dictionary representing the structure."""
    structure = {}
    for root, dirs, files in os.walk(local_path):
        relative_root = os.path.relpath(root, local_path)
        if relative_root == '.':
            relative_root = ''
        for name in files:
            file_path = os.path.join(root, name)
            relative_file_path = os.path.join(relative_root, name)
            structure[relative_file_path] = file_path # Store local path for files
    return structure

def get_file_content_local(file_path):
    """Fetches the content of a local file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"--- Could not read file: {file_path} ({e}) ---"


def build_context_from_repo_local(repo_structure, base_path):
    """Builds a context string from the organized local repository structure."""
    context = ""
    for relative_path, local_path in repo_structure.items():
        context += f"--- File: {relative_path} ---\n"
        context += get_file_content_local(local_path)
        context += "\n\n"
    return context


# Example usage:
repo_url = "https://github.com/raj-chinagundi/cnnscraper" # Replace with your GitHub repo URL
local_repo_path = clone_github_repo(repo_url)

if local_repo_path:
  repo_structure_local = organize_repo_structure_local(local_repo_path)
  repo_context = build_context_from_repo_local(repo_structure_local, local_repo_path)
  print(repo_structure_local)

Repository cloned to 'repo'
{'README.md': 'repo/README.md', 'datacnn.png': 'repo/datacnn.png', 'cnnscraper.py': 'repo/cnnscraper.py', 'requirement.txt': 'repo/requirement.txt', 'cnnscraper.png': 'repo/cnnscraper.png', '.git/shallow': 'repo/.git/shallow', '.git/config': 'repo/.git/config', '.git/HEAD': 'repo/.git/HEAD', '.git/packed-refs': 'repo/.git/packed-refs', '.git/description': 'repo/.git/description', '.git/index': 'repo/.git/index', '.git/objects/pack/pack-50fd9303793b07e4479b94c5258be6ae947e2b3d.idx': 'repo/.git/objects/pack/pack-50fd9303793b07e4479b94c5258be6ae947e2b3d.idx', '.git/objects/pack/pack-50fd9303793b07e4479b94c5258be6ae947e2b3d.pack': 'repo/.git/objects/pack/pack-50fd9303793b07e4479b94c5258be6ae947e2b3d.pack', '.git/hooks/pre-merge-commit.sample': 'repo/.git/hooks/pre-merge-commit.sample', '.git/hooks/push-to-checkout.sample': 'repo/.git/hooks/push-to-checkout.sample', '.git/hooks/pre-push.sample': 'repo/.git/hooks/pre-push.sample', '.git/hooks/fsmonitor-watchman.sa

In [ ]:
repo_context

'--- File: README.md ---\n# cnnscraper\n# CNN News Scraper  \n## Description \nCNN News Scraper is a Python application that allows you to scrape news articles from CNN\'s website. It provides an easy way to collect and analyze news data from CNN.  \n\n## Installation  \nTo install the required dependencies, run the following command:  \n```pip install -r requirement.txt```\n## Run the code\n```python cnnscraper.py```\n\n## Screenshots\n### Terminal Running Process\n![Terminal run](https://github.com/raj-chinagundi/cnnscraper/blob/main/cnnscraper.png)\n### Dataset\n![data](https://github.com/raj-chinagundi/cnnscraper/blob/main/datacnn.png)\n\n\n--- File: datacnn.png ---\n--- Could not read file: repo/datacnn.png (\'utf-8\' codec can\'t decode byte 0x89 in position 0: invalid start byte) ---\n\n--- File: cnnscraper.py ---\nimport requests\nfrom bs4 import BeautifulSoup\nimport re\nfrom datetime import datetime\nimport csv\nimport os\n\n#images directory and csv file\nfolder_name = "arti

In [ ]:
prompts={
    "cot":f"""Help me create MkDocs documentation for my Python project. Let me break this down step by step:

Step 1: First, analyze my project structure to understand what needs documenting
- What files and folders should I scan?
- What content can be extracted automatically?

Step 2: Plan the documentation structure
- How should I organize the MkDocs navigation?
- What sections are essential for a Python project?

Step 3: Identify automation opportunities
- Which parts can be generated from code comments?
- How can I extract API documentation automatically?

Step 4: Create the implementation plan
- What tools and scripts do I need?
- How can I make this repeatable for future projects?

Please walk me through each step with specific recommendations.

My Repo:
{repo_context}


""",
 "tot":f"""I need to automate MkDocs documentation for my Python project. Let me explore different approaches:

APPROACH A: Manual Setup with Templates
- Create standard MkDocs templates
- Manually organize content into sections
- Use mkdocstrings for API docs only

APPROACH B: Full Automation with Scripts
- Write Python scripts to scan project structure
- Auto-generate navigation from folder structure
- Extract all docstrings and comments automatically

APPROACH C: Hybrid Semi-Automated Approach
- Use automation for repetitive tasks (API docs, changelog)
- Manual curation for high-level content (tutorials, guides)
- Template-based structure with automated content filling

For each approach, evaluate:
1. Setup time and complexity
2. Maintenance overhead
3. Documentation quality
4. Flexibility for different project types

Which approach would you recommend and why?

My Repo:
{repo_context}
""",
    "meta":f"""You are a Senior Technical Writer and DevOps Engineer with expertise in documentation automation and Python tooling.

CONTEXT: I'm a Python developer working on multiple open-source projects. I spend too much time manually creating and updating documentation, and my current docs are inconsistent across projects.

PROJECT DETAILS:
- Python projects using different frameworks (Flask, FastAPI, Django)
- Mix of libraries and applications
- Team of 3-5 contributors per project
- Users range from beginners to advanced developers

CURRENT PAIN POINTS:
- Documentation gets outdated quickly
- Inconsistent structure across projects
- Time-consuming to set up new project docs
- API documentation often missing or incomplete

GOAL: Create a streamlined MkDocs automation workflow that can be applied to any Python project with minimal setup.

CONSTRAINTS:
- Must work with existing GitHub Actions CI/CD
- Should integrate with current development workflow
- Minimal learning curve for team members
- Budget-friendly (prefer open-source tools)

EXPECTED DELIVERABLE:
Please provide a complete automation strategy including:
1. Recommended MkDocs plugins and configuration
2. Step-by-step setup instructions
3. Automation scripts or tools needed
4. Maintenance workflow for keeping docs updated
5. Template structure that works across different project types

TONE: Practical and actionable - I want something I can implement this week, not theoretical advice.

My Repo:
{repo_context}
"""
}

In [ ]:
# prompt: select either of three prompts and set it as final prompt

prompt_choice = "meta" #@param ["cot", "tot", "meta"]
final_prompt = prompts[prompt_choice]
print(f"Selected prompt: {prompt_choice}")
print("\nFinal Prompt Content:")
final_prompt


Selected prompt: meta

Final Prompt Content:


'You are a Senior Technical Writer and DevOps Engineer with expertise in documentation automation and Python tooling.\n\nCONTEXT: I\'m a Python developer working on multiple open-source projects. I spend too much time manually creating and updating documentation, and my current docs are inconsistent across projects.\n\nPROJECT DETAILS:\n- Python projects using different frameworks (Flask, FastAPI, Django)\n- Mix of libraries and applications\n- Team of 3-5 contributors per project\n- Users range from beginners to advanced developers\n\nCURRENT PAIN POINTS:\n- Documentation gets outdated quickly\n- Inconsistent structure across projects  \n- Time-consuming to set up new project docs\n- API documentation often missing or incomplete\n\nGOAL: Create a streamlined MkDocs automation workflow that can be applied to any Python project with minimal setup.\n\nCONSTRAINTS:\n- Must work with existing GitHub Actions CI/CD\n- Should integrate with current development workflow\n- Minimal learning cur

In [ ]:
from pprint import pprint
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=final_prompt
)

In [ ]:
output_mkdocs_context=''.join(response.text)
print(output_mkdocs_context)

As a Senior Technical Writer and DevOps Engineer, I understand your pain points perfectly. Manual, inconsistent documentation is a productivity killer. The goal is a **streamlined, automated MkDocs workflow** that's adaptable across your Flask, FastAPI, Django, and library projects, leveraging your existing Python and GitHub Actions expertise.

Here's a practical, actionable strategy you can implement this week for your `cnnscraper` project, designed to be easily extensible to your other projects.

---

## Streamlined MkDocs Automation Workflow for Python Projects

This strategy focuses on automating API documentation, standardizing structure, and integrating seamlessly with GitHub Actions for effortless deployment.

### Core Tools & Philosophy

*   **MkDocs:** A fast, simple static site generator.
*   **Material for MkDocs:** The best theme for MkDocs, offering superb features, aesthetics, and search.
*   **mkdocstrings:** The powerhouse for auto-generating API documentation directly 

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional


class DocsDirectory(BaseModel):
    index_md: str = Field(..., alias="index.md")
    installation_md: str = Field(..., alias="installation.md")
    usage_md: str = Field(..., alias="usage.md")


class MkDocsProject(BaseModel):
    project_name: str
    mkdocs_yml: str = Field(..., alias="mkdocs.yml")
    docs: DocsDirectory
    code_dir: Optional[str] = Field(None, description="Directory with source code, e.g., 'src/' or 'my_module/'")
    readme_md: Optional[str] = Field(None, alias="README.md")

In [ ]:
response2 = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"""Use this as a context for generating my MkDocs Documentation, If similar files names are not present in output structure, semantically match the content and put the content under the respective md keys which matches the content. Context:{output_mkdocs_context}\n\n
    Note:
    1)Please don't use any unnecessary markdown extensions which are not available by default.
    2)Follow the below guidelines while generating mkdocs.yml

    **MkDocs YAML Generation Rule:**

    - Only use configuration options and extension parameters that are officially documented and supported by the current versions of MkDocs, mkdocs-material, and mkdocstrings.
    - Do not include any Python object references (e.g., `!!python/name:...`) or undocumented options in the YAML.
    - For mkdocstrings handler options, place any custom or extra options under the `extra:` key inside `options:`.
    - Do not add options that are deprecated, experimental, or not recognized by the relevant plugins.
    - If unsure about an option, omit it and use the default configuration.
    - Don't hallucinate add md file references which are not present in the context!!
    """,
    config={
        "response_mime_type": "application/json",
        "response_schema": MkDocsProject,
    },
)

In [ ]:
response2.text

'{\n  "project_name": "cnnscraper",\n  "mkdocs.yml": "site_name: CNN Scraper Documentation\\nsite_url: https://your-username.github.io/cnnscraper-docs/\\nrepo_url: https://github.com/raj-chinagundi/cnnscraper\\nrepo_name: raj-chinagundi/cnnscraper\\nedit_uri: blob/main/docs/\\n\\ntheme:\\n  name: material\\n  features:\\n    - navigation.tabs\\n    - navigation.sections\\n    - toc.integrate\\n    - navigation.path\\n    - search.suggest\\n    - search.highlight\\n    - content.tabs.link\\n    - content.code.annotation\\n    - content.code.copy\\n  palette:\\n    - media: \\"(prefers-color-scheme: light)\\"\\n      scheme: default\\n      toggle:\\n        icon: material/brightness-7\\n        name: Switch to dark mode\\n    - media: \\"(prefers-color-scheme: dark)\\"\\n      scheme: slate\\n      toggle:\\n        icon: material/brightness-4\\n        name: Switch to light mode\\n  icon:\\n    logo: material/robot-happy\\n  favicon: assets/favicon.png\\n\\nplugins:\\n  - search\\n  - 

In [ ]:
!rm -rf /content/cnnscraper

In [ ]:
import os
import json

# Assuming 'response2' contains the model output in the specified Pydantic structure
mkdocs_project_data = json.loads(response2.text)

project_name = mkdocs_project_data["project_name"].replace(" ","_")
mkdocs_yml_content = mkdocs_project_data["mkdocs.yml"]
docs_content = mkdocs_project_data["docs"]
code_dir = mkdocs_project_data.get("code_dir")
readme_md_content = mkdocs_project_data.get("README.md")

# Create the project directory
if not os.path.exists(project_name):
    os.makedirs(project_name)

# Create the mkdocs.yml file
with open(os.path.join(project_name, "mkdocs.yml"), "w") as f:
    f.write(mkdocs_yml_content)

# Create the docs directory
docs_path = os.path.join(project_name, "docs")
if not os.path.exists(docs_path):
    os.makedirs(docs_path)

# Create the markdown files in the docs directory
for file_name, content in docs_content.items():
    with open(os.path.join(docs_path, file_name), "w") as f:
        f.write(content)

# Create the code directory if specified
if code_dir and not os.path.exists(os.path.join(project_name, code_dir)):
    os.makedirs(os.path.join(project_name, code_dir))

# Create the README.md file if content exists
if readme_md_content:
    with open(os.path.join(project_name, "README.md"), "w") as f:
        f.write(readme_md_content)

print(f"MkDocs project '{project_name}' created successfully!")

MkDocs project 'cnnscraper' created successfully!


## On Local

build the documentation using the `mkdocs build` command, or serve it locally using `mkdocs serve`.

Before building, make sure you have MkDocs and the necessary plugins installed:

```pip install -q mkdocs mkdocs-material "mkdocstrings[python]" mkdocs-autorefs mkdocs-section-index mkdocs-literate-nav pymdown-extensions```

In [ ]:
from google.colab import files

# Specify the folder path you want to download
folder_path = 'cnnscraper' # Replace with the actual folder name

# Compress the folder
!zip -r {folder_path}.zip "{folder_path}"


  adding: cnnscraper/ (stored 0%)
  adding: cnnscraper/docs/ (stored 0%)
  adding: cnnscraper/docs/index.md (deflated 46%)
  adding: cnnscraper/docs/installation.md (deflated 50%)
  adding: cnnscraper/docs/usage.md (deflated 49%)
  adding: cnnscraper/README.md (deflated 46%)
  adding: cnnscraper/mkdocs.yml (deflated 60%)


## Homework:
- [deploy](https://www.youtube.com/watch?v=1NdfYh-QzZ0) documentation for your repository

- [few-shot-prompting](https://www.youtube.com/watch?v=sW5xoicq5TY) try prompt variations